In [2]:
!pip install pytrends


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install pytrends --upgrade

Requirement already up-to-date: pytrends in c:\programdata\anaconda3\lib\site-packages (4.7.3)


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [37]:
from pytrends.request import TrendReq

In [38]:
import pandas as pd
import time
import psycopg2 

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


In [40]:
df_raw = pd.read_excel("keytrends.xlsx", sheet_name='key_word')
#Thay the cot co ky tu / thanh _
df = df_raw.rename(columns=lambda b: b.replace("/ ","_"))
df.head()

,news,person,film,elearning_online,diseases,songs,travel
0,Bầu cử Tổng thống Mỹ,Nguyễn Đức Chung,Ký Sinh Trùng,Zoom,Coronavirus tips,Anh Thanh Niên,Du lịch Cát Bà
1,Covid 19,Đường Nhuệ,Pháp Sư Mù,Vnedu,Covid 19,Hoa Nở Không Màu,Du lịch Đà Lạt
2,Tuấn 'khỉ',Donald Trump,Mắt Biếc,Smas,Triệu chứng corona,Buồn Làm Chi Em Ơi,Du lịch Sapa
3,iPhone 12,Joe Biden,Tiệc Trăng Máu,Viettelstudy,Bộ Y Tế,Sai Lầm Của Anh,Du lịch Đà Nẵng
4,Vụ án Hồ Duy Hải,Jack,One Piece Stampede,Google classroom,Khẩu trang n95,Hoa Hải Đường,Du lịch Thới Sơn


In [41]:
column_list = df.columns.ravel()
print(column_list)

['news' 'person' 'film' 'elearning_online' 'diseases' 'songs' 'travel']


In [50]:
# Tao file excel
df_db=[]
writer=pd.ExcelWriter('vn_trend_2020.xlsx', engine='xlsxwriter')
#lay du lieu tu google trend
pytrend = TrendReq(hl='vi-VN', tz=360, timeout=10)
for i in column_list:
    dataset = []
    keywords = list(df[i].dropna()) 
    for x in range(0,len(keywords)):
        keyword =[keywords[x]]
        pytrend.build_payload(
            kw_list=keyword,
            cat=0,
            timeframe='2020-01-01 2020-12-31',
            geo='VN',
            gprop=''
        )
        data=pytrend.interest_over_time()
        if not data.empty:
            data = data.drop(labels=['isPartial'],axis='columns')
            dataset.append(data)
    result=pd.concat(dataset,axis=1)
    result.to_excel(writer,sheet_name=i)
    
   #Tao dataframe cau truc giong voi DB 
    result.reset_index(level=0, inplace=True)
    df_melt=pd.melt(result, id_vars=["date"],var_name="Keywords", value_name="Value")
    df_melt['Trend_type'] = i   
    #result_final=pd.concat([df_melt],ignore_index=True)
    df_db.append(df_melt)
    
#Save excel 
writer.save()
df_db_final = pd.concat(df_db)
df_db_final.to_csv('all.csv')


<h1> Insert dữ liệu vào postgresql và đưa kết quả top 10 keyword ra file</h1>

CREATE TABLE public.vn_trending
(
    "ID" integer NOT NULL DEFAULT nextval('"trending_ID_seq"'::regclass),
    date date,
    keywords character varying COLLATE pg_catalog."default",
    value bigint,
    trend_type character varying COLLATE pg_catalog."default",
    CONSTRAINT trending_pkey PRIMARY KEY ("ID")
)


In [52]:
import psycopg2.extras as extras

In [65]:

try:
    
    print('Connecting to the PostgreSQL database...')
    conn=psycopg2.connect(
        host="localhost",
        database="GoogleTrend",
        user="postgres",
        password="Qu@ng@nh")
    
    cursor = conn.cursor()
    
    #truncate table
    cursor.execute('truncate table vn_trending')
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_db_final.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_db_final.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES %%s" % ("vn_trending", cols)

    extras.execute_values(cursor, sql, tpls)
    conn.commit()
    print("Data inserted using execute_values() successfully..")

   

     ##select top 10
    # declare a cursor object from the connection

    sql_top = "select keywords, sum(Value) as sum_Value from vn_trending group by keywords order by sum(Value) desc limit 10"
    cursor.execute(sql_top)
    # Fetch all the records
    tuples = cursor.fetchall()
    # Preparing list of columns for dataframe
    df_top = pd.DataFrame(tuples,columns=['keywords','sum_Value'])
    print(df_top.head(100))
    ##Ghi ra excel
    df_top.to_excel('vn_trending_result.xls')
    
    # close the connection object
    conn.close()

        
        
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

Connecting to the PostgreSQL database...
Data inserted using execute_values() successfully..
             keywords sum_Value
0                Jack      3441
1       Du lịch Sapa       2303
2          Tiêu Chiến      2160
3    Du lịch Mộc Châu      2060
4  Buồn Làm Chi Em Ơi      1993
5     Du lịch Đà Lạt       1777
6  Du lịch Nha Trang       1687
7      Anh Thanh Niên      1666
8       Bánh Mì Không      1627
9             Bộ Y Tế      1615
Database connection closed.
